In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from utils import load_data, fill_empty, find_redundant_features
from scipy.stats import chi2_contingency

# Loading data

In [2]:
TRAINING_DATA = "cup98LRN.csv"

In [3]:
raw_data = load_data(TRAINING_DATA)

In [4]:
raw_data

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,,,0,0,,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,,,5001,0,,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,,,3801,0,,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,,,4005,0,X,...,18.0,1,L,4,F,X,X,X,11.0,A


# Data cleaning

In [5]:
without_duplicates = raw_data.drop_duplicates()

In [6]:
filled_data = fill_empty(without_duplicates)

In [7]:
filled_data

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,,,0,0,,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,,,5001,0,,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,,,3801,0,,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,,,4005,0,X,...,18.0,1,L,4,F,X,X,X,11.0,A


In [8]:
filled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95412 entries, 0 to 95411
Columns: 481 entries, ODATEDW to GEOCODE2
dtypes: float64(97), int64(310), object(74)
memory usage: 350.9+ MB


# Feature selection

In [9]:
training_data = filled_data.drop(["TARGET_D"], axis=1)

## Categorical 

In [10]:
object_columns = training_data.select_dtypes(include=['object']).columns

In [11]:
categorical_features = filled_data[object_columns]

In [12]:
categorical_features = categorical_features.replace({" ": "-"})
categorical_features

,OSOURCE,STATE,ZIP,MAILCODE,PVASTATE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,...,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,-,-,0,-,-,-,-,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,-,-,0,-,-,-,-,...,N1E,N1E,-,F1E,L,G,X,X,X,A
2,AMH,NC,27017,-,-,0,-,-,-,-,...,-,S4D,S4D,S3D,L,E,X,X,X,C
3,BRY,CA,95953,-,-,0,-,-,-,-,...,A1D,A1D,-,-,L,E,X,X,X,C
4,-,FL,33176,-,-,0,X,X,-,-,...,A3D,I4E,A3D,A3D,L,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,ASE,AK,99504,-,-,0,-,-,-,-,...,-,-,-,-,L,G,X,X,X,C
95408,DCD,TX,77379,-,-,0,-,-,-,-,...,-,-,-,-,L,F,X,X,X,A
95409,MBC,MI,48910,-,-,0,-,X,-,-,...,P1D,P1D,-,-,L,E,X,X,X,B
95410,PRV,CA,91320,-,-,0,X,-,-,-,...,A1F,A1F,S2F,S3F,L,F,X,X,X,A


In [13]:
categorical_features["TARGET"] = raw_data["TARGET_B"]

In [14]:
def get_dependent_features(df, target_value, alpha=0.05):
    dependent_columns = []
    features = df.columns.to_list()
    features.remove(target_value)
    
    for column in features:
        _, p, _, _ = chi2_contingency(pd.crosstab(index=df[target_value], columns=df[column]).to_numpy())
        if p <= alpha:
            dependent_columns.append(column)
    
    return dependent_columns

In [15]:
dependent_columns = get_dependent_features(categorical_features, "TARGET")

In [16]:
categorical_features_important = categorical_features[dependent_columns]
categorical_features_important

,OSOURCE,STATE,MAILCODE,RECINHSE,RECP3,DOMAIN,CLUSTER,HOMEOWNR,GENDER,SOLP3,...,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2A
0,GRI,IL,-,-,-,T2,36,-,F,-,...,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,E
1,BOA,CA,-,-,-,S1,14,H,M,-,...,L1E,-,-,N1E,N1E,N1E,N1E,-,F1E,G
2,AMH,NC,-,-,-,R2,43,U,M,-,...,S4F,-,S4D,S4D,-,-,S4D,S4D,S3D,E
3,BRY,CA,-,-,-,R2,44,U,F,-,...,S4E,S2D,S2D,A1D,A1D,A1D,A1D,-,-,E
4,-,FL,-,X,X,S2,16,H,F,-,...,L3D,A2D,A2D,A3D,A3D,A3D,I4E,A3D,A3D,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,ASE,AK,-,-,-,C2,27,-,M,-,...,-,-,-,-,-,-,-,-,-,G
95408,DCD,TX,-,-,-,C1,24,H,M,-,...,-,-,-,-,-,-,-,-,-,F
95409,MBC,MI,-,-,X,C3,30,-,M,-,...,F1D,-,F1D,-,-,P1D,P1D,-,-,E
95410,PRV,CA,-,X,-,C1,24,H,F,-,...,S3F,S2F,S2F,A1F,A1F,A1F,A1F,S2F,S3F,F


In [17]:
categorical_features_important_one_hot = pd.get_dummies(categorical_features_important)

In [18]:
categorical_features_important_one_hot["TARGET"] = raw_data["TARGET_B"]

In [19]:
dependent_columns = get_dependent_features(categorical_features_important_one_hot, "TARGET")

In [20]:
categorical_features_selected = categorical_features_important_one_hot[dependent_columns]

## Numerical

In [21]:
numerical_columns = training_data.select_dtypes(include=['int64', 'float64']).columns

In [22]:
numerical_features = filled_data[numerical_columns]

In [23]:
numerical_features

,ODATEDW,TCODE,DOB,AGE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,...,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.000000,1.527773,3.886248,5.345699,0,0.152075,0.059166,...,9512,8911,9003.000000,4.000000,7.741935,95515,0,0,4,39.0
1,9401,1,5202,46.000000,1.000000,6.000000,9.000000,16,0.000000,0.000000,...,9512,9310,9504.000000,18.000000,15.666667,148535,0,0,2,1.0
2,9001,1,0,61.611649,1.527773,3.000000,1.000000,2,0.000000,0.000000,...,9512,9001,9101.000000,12.000000,7.481481,15078,0,1,4,60.0
3,8701,0,2801,70.000000,1.527773,1.000000,4.000000,2,0.000000,0.000000,...,9512,8702,8711.000000,9.000000,6.812500,172556,0,1,4,41.0
4,8601,0,2001,78.000000,1.000000,3.000000,2.000000,60,1.000000,0.000000,...,9601,7903,8005.000000,14.000000,6.864865,7112,0,1,2,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,1,0,61.611649,1.527773,3.886248,5.345699,0,0.152075,0.059166,...,9602,9602,9151.022917,8.093739,25.000000,184568,0,0,1,12.0
95408,9601,1,5001,48.000000,1.000000,7.000000,9.000000,1,0.000000,0.000000,...,9603,9603,9151.022917,8.093739,20.000000,122706,0,1,1,2.0
95409,9501,1,3801,60.000000,1.527773,3.886248,5.345699,0,0.152075,0.059166,...,9610,9410,9501.000000,3.000000,8.285714,189641,0,1,3,34.0
95410,8601,0,4005,58.000000,1.527773,7.000000,5.345699,0,0.152075,0.059166,...,9701,8612,8704.000000,4.000000,12.146341,4693,1,1,4,11.0


In [24]:
corr_matrix = numerical_features.corr(method='pearson')

In [25]:
corr_matrix

,ODATEDW,TCODE,DOB,AGE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,...,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,HPHONE_D,RFA_2F,CLUSTER2
ODATEDW,1.000000,-0.033214,0.099450,-0.265999,0.011992,0.081450,0.045445,-0.070752,-0.012176,-0.025678,...,0.026900,0.976866,0.868699,-0.078963,0.216869,0.079036,-0.032270,-0.073435,-0.065082,-0.060136
TCODE,-0.033214,1.000000,-0.003527,0.009662,0.001381,-0.005071,-0.004569,-0.002897,-0.000731,-0.003981,...,0.001691,-0.033189,-0.028842,0.004956,-0.005778,-0.001920,0.004941,-0.002025,0.007835,0.001131
DOB,0.099450,-0.003527,1.000000,-0.679248,0.042862,0.196965,0.081192,0.023451,0.008830,-0.007219,...,-0.040782,0.098443,0.109837,0.019413,0.020828,-0.023648,-0.001703,0.076916,-0.048517,-0.089796
AGE,-0.265999,0.009662,-0.679248,1.000000,-0.060705,-0.217127,-0.081003,0.115603,0.005810,0.017893,...,0.046000,-0.264007,-0.254353,-0.007321,-0.063872,-0.024782,0.007922,0.083355,0.050085,0.049102
NUMCHLD,0.011992,0.001381,0.042862,-0.060705,1.000000,0.008071,0.008820,0.001156,0.009571,-0.002331,...,0.004595,0.011402,0.016696,-0.002965,-0.005604,0.001480,-0.005123,0.008842,0.007142,-0.007146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CONTROLN,0.079036,-0.001920,-0.023648,-0.024782,0.001480,0.057227,-0.019336,0.000420,-0.001171,-0.011762,...,-0.041290,0.088013,0.077896,0.018204,0.011507,1.000000,0.013165,-0.108747,-0.016545,-0.143244
TARGET_B,-0.032270,0.004941,-0.001703,0.007922,-0.005123,0.016425,0.008252,0.006189,0.009001,-0.000497,...,0.041158,-0.032701,-0.024770,-0.010260,-0.032444,0.013165,1.000000,-0.002626,0.072311,-0.016390
HPHONE_D,-0.073435,-0.002025,0.076916,0.083355,0.008842,-0.025190,-0.010760,0.213498,0.009925,0.007290,...,0.006471,-0.071569,-0.061333,0.008395,-0.044323,-0.108747,-0.002626,1.000000,0.007607,-0.018303
RFA_2F,-0.065082,0.007835,-0.048517,0.050085,0.007142,-0.070112,-0.038266,-0.000448,0.009740,0.003812,...,0.242880,-0.068630,0.042276,-0.127265,-0.277168,-0.016545,0.072311,0.007607,1.000000,0.043029


In [26]:
redundant_features = find_redundant_features(corr_matrix)

In [27]:
numerical_features

,ODATEDW,TCODE,DOB,AGE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,...,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.000000,1.527773,3.886248,5.345699,0,0.152075,0.059166,...,9512,8911,9003.000000,4.000000,7.741935,95515,0,0,4,39.0
1,9401,1,5202,46.000000,1.000000,6.000000,9.000000,16,0.000000,0.000000,...,9512,9310,9504.000000,18.000000,15.666667,148535,0,0,2,1.0
2,9001,1,0,61.611649,1.527773,3.000000,1.000000,2,0.000000,0.000000,...,9512,9001,9101.000000,12.000000,7.481481,15078,0,1,4,60.0
3,8701,0,2801,70.000000,1.527773,1.000000,4.000000,2,0.000000,0.000000,...,9512,8702,8711.000000,9.000000,6.812500,172556,0,1,4,41.0
4,8601,0,2001,78.000000,1.000000,3.000000,2.000000,60,1.000000,0.000000,...,9601,7903,8005.000000,14.000000,6.864865,7112,0,1,2,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,1,0,61.611649,1.527773,3.886248,5.345699,0,0.152075,0.059166,...,9602,9602,9151.022917,8.093739,25.000000,184568,0,0,1,12.0
95408,9601,1,5001,48.000000,1.000000,7.000000,9.000000,1,0.000000,0.000000,...,9603,9603,9151.022917,8.093739,20.000000,122706,0,1,1,2.0
95409,9501,1,3801,60.000000,1.527773,3.886248,5.345699,0,0.152075,0.059166,...,9610,9410,9501.000000,3.000000,8.285714,189641,0,1,3,34.0
95410,8601,0,4005,58.000000,1.527773,7.000000,5.345699,0,0.152075,0.059166,...,9701,8612,8704.000000,4.000000,12.146341,4693,1,1,4,11.0


In [28]:
numerical_features = numerical_features.drop(redundant_features, axis=1)
numerical_features

,ODATEDW,TCODE,DOB,AGE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,...,LASTGIFT,LASTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.000000,1.527773,3.886248,5.345699,0,0.152075,0.059166,...,10.0,9512,9003.000000,4.000000,7.741935,95515,0,0,4,39.0
1,9401,1,5202,46.000000,1.000000,6.000000,9.000000,16,0.000000,0.000000,...,25.0,9512,9504.000000,18.000000,15.666667,148535,0,0,2,1.0
2,9001,1,0,61.611649,1.527773,3.000000,1.000000,2,0.000000,0.000000,...,5.0,9512,9101.000000,12.000000,7.481481,15078,0,1,4,60.0
3,8701,0,2801,70.000000,1.527773,1.000000,4.000000,2,0.000000,0.000000,...,10.0,9512,8711.000000,9.000000,6.812500,172556,0,1,4,41.0
4,8601,0,2001,78.000000,1.000000,3.000000,2.000000,60,1.000000,0.000000,...,15.0,9601,8005.000000,14.000000,6.864865,7112,0,1,2,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,1,0,61.611649,1.527773,3.886248,5.345699,0,0.152075,0.059166,...,25.0,9602,9151.022917,8.093739,25.000000,184568,0,0,1,12.0
95408,9601,1,5001,48.000000,1.000000,7.000000,9.000000,1,0.000000,0.000000,...,20.0,9603,9151.022917,8.093739,20.000000,122706,0,1,1,2.0
95409,9501,1,3801,60.000000,1.527773,3.886248,5.345699,0,0.152075,0.059166,...,10.0,9610,9501.000000,3.000000,8.285714,189641,0,1,3,34.0
95410,8601,0,4005,58.000000,1.527773,7.000000,5.345699,0,0.152075,0.059166,...,18.0,9701,8704.000000,4.000000,12.146341,4693,1,1,4,11.0


In [29]:
corr_matrix2 = numerical_features.corr(method='pearson')
corr_matrix_target = corr_matrix2["TARGET_B"].sort_values(ascending=False)
corr_matrix_target = abs(corr_matrix2["TARGET_B"].sort_values(ascending=False))
corr_matrix_target

TARGET_B    1.000000
RFA_2F      0.072311
NGIFTALL    0.050896
LASTDATE    0.041158
CARDPROM    0.032467
              ...   
ODATEDW     0.032270
AVGGIFT     0.032444
LASTGIFT    0.035526
ADATE_5          NaN
ADATE_15         NaN
Name: TARGET_B, Length: 346, dtype: float64

In [30]:
numerical_features = numerical_features[list(corr_matrix_target.index)]
numerical_features = numerical_features.drop(["ADATE_5", "ADATE_15", "TARGET_B"], axis=1)

In [31]:
numerical_features

,RFA_2F,NGIFTALL,LASTDATE,CARDPROM,HV1,HVP3,DMA,HVP5,IC5,HHAS3,...,NEXTDATE,RAMNT_14,MAXRDATE,RAMNT_12,RAMNT_8,RAMNT_16,MINRAMNT,ODATEDW,AVGGIFT,LASTGIFT
0,4,31,9512,27,479,13,682.0,47,12883,51,...,9003.000000,10.000000,9402,14.859901,15.673702,11.000000,5.0,8901,7.741935,10.0
1,2,3,9512,12,5468,98,803.0,99,36175,66,...,9504.000000,13.252321,9512,14.859901,15.673702,12.000000,10.0,9401,15.666667,25.0
2,4,27,9512,26,497,6,518.0,50,11576,31,...,9101.000000,9.000000,9207,11.000000,15.673702,9.000000,2.0,9001,7.481481,5.0
3,4,16,9512,27,1000,50,862.0,92,15130,26,...,8711.000000,10.000000,9411,10.000000,15.673702,7.000000,2.0,8701,6.812500,10.0
4,2,37,9601,43,576,2,528.0,67,9836,11,...,8005.000000,10.000000,9601,14.859901,15.000000,14.032142,3.0,8601,6.864865,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1,1,9602,6,988,49,743.0,92,18807,70,...,9151.022917,13.252321,9602,14.859901,15.673702,14.032142,25.0,9601,25.000000,25.0
95408,1,1,9603,4,1679,88,618.0,97,26538,61,...,9151.022917,13.252321,9603,14.859901,15.673702,14.032142,20.0,9601,20.000000,20.0
95409,3,7,9610,14,376,0,551.0,9,12178,26,...,9501.000000,13.252321,9501,10.000000,10.000000,14.032142,3.0,9501,8.285714,10.0
95410,4,41,9701,36,2421,99,803.0,99,15948,46,...,8704.000000,13.252321,9608,14.859901,15.673702,18.000000,5.0,8601,12.146341,18.0


# Saving data

In [32]:
categorical_features_selected.to_csv("categorical_features.csv")

In [33]:
numerical_features.to_csv("numerical_features.csv")

In [34]:
raw_data["TARGET_B"].to_csv("targets.csv")